# Documentation 
https://github.com/innovata/DSE-REST-API  

# [1] 3DEXPERIENCE Platform CLM Agent 생성  
- "F:\pypjts\DSE-REST-API\How to use APIs\credentials"경로에 Agent ID, Password 저장 

# [2] 파이썬 가상환경(Python Virtual Environment) 생성 및 dserestapi 패키지 설치



In [ ]:
# # 개발용 경로설정
# import os, sys 
# sys.path.append(r"F:\pypjts\DSE-REST-API\src")

In [ ]:
# 커맨드 프롬프트 사용 권장
# !pip install dserestapi

# **********************************************************************

# [3.1] 환경셋업 | Normal Case: 일반유저 (Member) 계정으로 만든 에이전트 사용

In [2]:
import os
import re 
os.environ["CLM_AGENT_CREDENTIAL_PATH"] = r"F:\pypjts\DSE-REST-API\How to use APIs\credentials\Member\CML Agent ID-PW.json"
os.environ["3DX_PLATFORM_TENANT_URI"] = "https://r1132100527066-apk2-sgi.3dexperience.3ds.com:443"
for k, v in os.environ.items():
    if re.search(r"^CLM|^3DX", k):
        print([k, v])

['CLM_AGENT_CREDENTIAL_PATH', 'F:\\pypjts\\DSE-REST-API\\How to use APIs\\credentials\\Member\\CML Agent ID-PW.json']
['3DX_PLATFORM_TENANT_URI', 'https://r1132100527066-apk2-sgi.3dexperience.3ds.com:443']


In [3]:
import dserestapi 
dserestapi

<module 'dserestapi' from 'f:\\pypjts\\DSE-REST-API\\.env64\\Lib\\site-packages\\dserestapi\\__init__.py'>

# [3.2] 환경셋업 | Error Case: 어드민 계정으로 만든 에이전트 사용 

In [1]:
# import os
# import re 
# os.environ["CLM_AGENT_CREDENTIAL_PATH"] = r"F:\pypjts\DSE-REST-API\How to use APIs\credentials\Admin\CML Agent ID-PW.json"
# os.environ["3DX_PLATFORM_TENANT_URI"] = "https://r1132100527066-apk2-sgi.3dexperience.3ds.com:443"
# from dserestapi import Storages
# api = Storages()
# res = api.get()
# res.json() 

# **********************************************************************

# [4] 사용 예제 | Storages

### [4.1] 스토리지 목록 가져오기 

In [4]:
# 모든 스토리지 리스트 가져오기
from dserestapi import Storages
api = Storages()
res = api.get()
res

<Response [200]>

In [5]:
import json 
file = os.path.join(os.getcwd(), "01_Storages", "Storages List.json")
os.makedirs(os.path.dirname(file), exist_ok=True)
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

In [ ]:
# !pip install pandas 

In [6]:
import pandas as pd 
df = pd.DataFrame(res.json()['cards'])
df#.head(5)

,id,resourceId,projectId,resourceUUID,workspaceId,name,creator,kind,type,created,lastModified,permissions
0,0add512f-3759-487b-aa40-9c5019874ea3,20241217_16:00_dsjeon_storage,dp-global-000000,0add512f-3759-487b-aa40-9c5019874ea3,dw-global-000000-default,20241217_16:00_dsjeon_storage,jeondongsoo,Storage,ObjectStorage,1734418711011,1734418711007,"{'read': True, 'write': True, 'execute': True}"
1,7118465e-739b-4746-8fc7-657755764ee7,240129_part_information_OU3,dp-global-000000,7118465e-739b-4746-8fc7-657755764ee7,dw-global-000000-default,240129_part_information_OU3,N/A,Storage,IndexUnit,1707553208946,1745657169549,"{'read': True, 'write': True, 'execute': True}"
2,R1132100527066_3DS_AdditiveManufacturingQualit...,3DS_AdditiveManufacturingQualityAnalyst_Data,ca221e4a-46b1-43b1-9c3e-4eea8a0aa662,R1132100527066_3DS_AdditiveManufacturingQualit...,ca221e4a-46b1-43b1-9c3e-4eea8a0aa662,3DS_AdditiveManufacturingQualityAnalyst_Data,svc_sgi_clouduser,Storage,ObjectStorage,1751697876373,1751697876334,"{'read': True, 'write': True, 'execute': True}"
3,R1132100527066_3DS_AdditiveManufacturingQualit...,3DS_AdditiveManufacturingQualityAnalyst_Image,ca221e4a-46b1-43b1-9c3e-4eea8a0aa662,R1132100527066_3DS_AdditiveManufacturingQualit...,ca221e4a-46b1-43b1-9c3e-4eea8a0aa662,3DS_AdditiveManufacturingQualityAnalyst_Image,svc_sgi_clouduser,Storage,ObjectStorage,1751697876576,1751697876537,"{'read': True, 'write': True, 'execute': True}"
4,R1132100527066_3DS_AdditiveManufacturingQualit...,3DS_AdditiveManufacturingQualityAnalyst_Result,ca221e4a-46b1-43b1-9c3e-4eea8a0aa662,R1132100527066_3DS_AdditiveManufacturingQualit...,ca221e4a-46b1-43b1-9c3e-4eea8a0aa662,3DS_AdditiveManufacturingQualityAnalyst_Result,svc_sgi_clouduser,Storage,ObjectStorage,1751697876473,1751697876435,"{'read': True, 'write': True, 'execute': True}"
...,...,...,...,...,...,...,...,...,...,...,...,...
152,5841fd29-a3a3-4780-8343-b1fe0ccaae71,r2vsurvey,dp-r2vsurvey-000000,5841fd29-a3a3-4780-8343-b1fe0ccaae71,dw-r2vsurvey-000000-default,r2vsurvey,svc_sgi_clouduser,Storage,IndexUnit,1745666684450,1751707573630,"{'read': True, 'write': False, 'execute': False}"
153,3fbd978d-c9ce-43af-9f73-ea4aed3ab4b5,sgi,dp-global-000000,3fbd978d-c9ce-43af-9f73-ea4aed3ab4b5,dw-global-000000-default,sgi,jihunyoon,Storage,IndexUnit,1734678234116,1734678234102,"{'read': True, 'write': True, 'execute': True}"
154,b35600d0-6c8d-4142-9e48-d35ab335d62b,sgi_test_syyou,dp-global-000000,b35600d0-6c8d-4142-9e48-d35ab335d62b,dw-global-000000-default,sgi_test_syyou,mobilevm,Storage,IndexUnit,1734483191336,1734483191331,"{'read': True, 'write': True, 'execute': True}"
155,595464cb-65bd-44d9-b719-c08b1a605070,syyou_storage,dp-global-000000,595464cb-65bd-44d9-b719-c08b1a605070,dw-global-000000-default,syyou_storage,mobilevm,Storage,ObjectStorage,1734677241229,1734677241224,"{'read': True, 'write': True, 'execute': True}"


### [4.2] ObjectStorage 신규 생성 

In [7]:
# 스토리지 생성
res = api.create(
    stype="ObjectStorage",
    name="DemoObject.CreatedByRestAPI",
    description="테스트 후 삭제할 것"
)
res 

<Response [201]>

In [8]:
file = os.path.join(os.getcwd(), "01_Storages", "Create A Storage.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

### [4.3] 생성한 스토리지 찾기

In [9]:
# 스토리지 검색-1
res = api.search_by_name(
    name="DemoObject.CreatedByRestAPI",
    # workspace_id="dw-global-000000-default"
)
res 

<Response [200]>

In [10]:
file = os.path.join(os.getcwd(), "01_Storages", "Search-1 the Storage.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

In [12]:
from datetime import datetime 
datetime.fromtimestamp(1754029975886/1000)

datetime.datetime(2025, 8, 1, 15, 32, 55, 886000)

### [4.4] 스토리지 비우기 
- 샘플파일 아무거나 수동으로 업로드

In [13]:
resource_uuid = "5094f94a-ad12-496b-bfa8-120abb874fc0"

In [14]:
res = api.clear(resource_uuid)
res 

<Response [204]>

- res.json() 없음 --> 결과는 플랫폼에서 직접 확인  

### [4.5] 스토리지 Export

In [15]:
res = api.export_(resource_uuid)
res 

<Response [200]>

In [16]:
file = os.path.join(os.getcwd(), "01_Storages", "Export the Storage.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

### [4.6] 스토리지 Import

In [17]:
payload = {
    "name": "DemoObject.CreatedByRestAPI",
    "identifier": "DemoObject.CreatedByRestAPI",
    "update": True,
    "content": "ewogICJzdG9yYWdlIiA6IHsKICAgICJAY2xhc3MiIDogIk9iamVjdFN0b3JhZ2UiLAogICAgInJlc291cmNlSWQiIDogIkRlbW9PYmplY3QuQ3JlYXRlZEJ5UmVzdEFQSSIsCiAgICAibmFtZSIgOiAiRGVtb09iamVjdC5DcmVhdGVkQnlSZXN0QVBJIiwKICAgICJkZXNjcmlwdGlvbiIgOiAi7YWM7Iqk7Yq4IO2bhCDsgq3soJztlaAg6rKDIiwKICAgICJraW5kIiA6ICJTdG9yYWdlIiwKICAgICJ0eXBlIiA6ICJPYmplY3RTdG9yYWdlIiwKICAgICJyZXNvdXJjZVBhdGgiIDogIlN0b3JhZ2UvRGVtb09iamVjdC5DcmVhdGVkQnlSZXN0QVBJLmpzb24iLAogICAgImNvbmZpZyIgOiB7CiAgICAgICJidWNrZXQiIDogImRlbW9vYmplY3RjcmVhdGVkYnlyZXN0YXBpIiwKICAgICAgImRlbGVnYXRpb25zIiA6IFsgewogICAgICAgICJuYW1lIiA6ICJTZW1hbnRpY0dyYXBoSW5kZXguRE9TQWNjb3VudC4yIiwKICAgICAgICAicGVybWlzc2lvbiIgOiAicmVhZCIKICAgICAgfSwgewogICAgICAgICJuYW1lIiA6ICJTZW1hbnRpY0dyYXBoSW5kZXguRE9TQWNjb3VudC4yIiwKICAgICAgICAicGVybWlzc2lvbiIgOiAid3JpdGUiCiAgICAgIH0sIHsKICAgICAgICAibmFtZSIgOiAiUFBFUy5Eb3NBY2NvdW50IiwKICAgICAgICAicGVybWlzc2lvbiIgOiAiZXhlY3V0ZSIKICAgICAgfSwgewogICAgICAgICJuYW1lIiA6ICJEYXRhc2V0R292ZXJuYW5jZSIsCiAgICAgICAgInBlcm1pc3Npb24iIDogInJlYWQiCiAgICAgIH0gXQogICAgfQogIH0sCiAgInZlcnNpb24iIDogewogICAgInZhbHVlIiA6ICIxLjIiCiAgfQp9"
}

In [19]:
res = api.import_(payload)
res.json()

{'@class': 'ObjectStorage',
 'resourceId': 'DemoObject.CreatedByRestAPI',
 'name': 'DemoObject.CreatedByRestAPI',
 'description': '테스트 후 삭제할 것',
 'kind': 'Storage',
 'type': 'ObjectStorage',
 'resourcePath': 'Storage/DemoObject.CreatedByRestAPI.json',
 'resourceUUID': '466733dc-501d-4b1f-a97e-295de4306e32',
 'projectId': 'dp-global-000000',
 'workspaceId': 'dw-global-000000-default',
 'tenant': 'R1132100527066',
 'created': '2025-08-01T02:14:19.356457Z',
 'modified': '2025-08-01T06:39:28.105776Z',
 'creator': 'jle69_gmail',
 'ceStamp': '68ed8d26-1d5e-49c9-abf0-12e973548f40',
 'id': '466733dc-501d-4b1f-a97e-295de4306e32',
 'config': {'bucket': 'demoobjectcreatedbyrestapi',
  'delegations': [{'name': 'SemanticGraphIndex.DOSAccount.2',
    'permission': 'read'},
   {'name': 'SemanticGraphIndex.DOSAccount.2', 'permission': 'write'},
   {'name': 'PPES.DosAccount', 'permission': 'execute'},
   {'name': 'DatasetGovernance', 'permission': 'read'}]}}

In [20]:
file = os.path.join(os.getcwd(), "01_Storages", "Import the Storage.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

- resourceUUID BEFORE/AFTER 비교

### [4.7] 생성한 스토리지 삭제

In [21]:
# API를 이용해서 재검색
res = api.search_by_name(
    name="DemoObject.CreatedByRestAPI"
)
resource_uuid = res.json()["cards"][0]["resourceUUID"]
resource_uuid

'466733dc-501d-4b1f-a97e-295de4306e32'

In [22]:
res = api.delete(resource_uuid=resource_uuid)
res 

<Response [200]>

- res.json() 없음 --> 플랫폼에서 확인

# **********************************************************************

# [5] 사용 예제 | Object Storage

### [5.1] Object Storage에 파일 업로드 

In [23]:
# 신규 스토리지 재생성 
from dserestapi import Storages
api = Storages()
res = api.create(
    stype="ObjectStorage",
    name="DemoObject.CreatedByRestAPI",
    description="테스트 후 바로 삭제할 것"
)
res.json()

{'@class': 'ObjectStorage',
 'resourceId': 'DemoObject.CreatedByRestAPI',
 'name': 'DemoObject.CreatedByRestAPI',
 'description': '테스트 후 바로 삭제할 것',
 'kind': 'Storage',
 'type': 'ObjectStorage',
 'resourcePath': 'Storage/DemoObject.CreatedByRestAPI.json',
 'resourceUUID': 'bd633137-0599-4686-83e1-09a160b5358e',
 'projectId': 'dp-global-000000',
 'workspaceId': 'dw-global-000000-default',
 'tenant': 'R1132100527066',
 'created': '2025-08-01T02:14:19.356457Z',
 'modified': '2025-08-01T06:43:10.342536Z',
 'creator': 'jle69_gmail',
 'ceStamp': '731b9802-9585-417d-bb97-878d1e34860c',
 'id': 'bd633137-0599-4686-83e1-09a160b5358e',
 'config': {'delegations': [{'name': 'SemanticGraphIndex.DOSAccount.2',
    'permission': 'read'},
   {'name': 'SemanticGraphIndex.DOSAccount.2', 'permission': 'write'},
   {'name': 'PPES.DosAccount', 'permission': 'execute'},
   {'name': 'DatasetGovernance', 'permission': 'read'}]}}

In [24]:
# 생성한 스토리지의 resourceUUID 추출 
resource_uuid = res.json()["resourceUUID"]
resource_uuid

'bd633137-0599-4686-83e1-09a160b5358e'

#### [5.1.1] 파일 업로드 | 경로 미지정
- 업로드할 파일은 항상 파일 절대경로를 사용해야함

In [25]:
from dserestapi import ObjectStorage
api = ObjectStorage()
upload_file = r"F:\pypjts\DSE-REST-API\How to use APIs\01_Storages\Export the Storage.json"
res = api.upload(resource_uuid=resource_uuid, file=upload_file, path=None)
res 

<Response [200]>

#### [5.1.2] 파일 업로드 | 경로 강제 지정

In [26]:
api = ObjectStorage()
upload_file = r"F:\pypjts\DSE-REST-API\How to use APIs\01_Storages\Import the Storage.json"
res = api.upload(resource_uuid=resource_uuid, file=upload_file, path="Manual_Path/")
res 

<Response [200]>

- JSON 데이터 없음

### [5.2] Object Storage에 디렉토리 내의 모든 파일들을 한번에 업로드 

In [27]:
# 업로드할 디렉토리 확인
import os 
top_dir = r"E:\__ENGINEER_DRIVE__\__DATASET__\__Dummy Data__"
upload_files = []
for root, dirs, files in os.walk(top_dir):
    for file in files:
        upload_files.append(os.path.join(root, file))

print(len(upload_files))
upload_files

50


['E:\\__ENGINEER_DRIVE__\\__DATASET__\\__Dummy Data__\\Basic Data Types\\all_dtypes.csv',
 'E:\\__ENGINEER_DRIVE__\\__DATASET__\\__Dummy Data__\\Basic Data Types\\json_list.json',
 'E:\\__ENGINEER_DRIVE__\\__DATASET__\\__Dummy Data__\\Basic Data Types\\_Boolean.csv',
 'E:\\__ENGINEER_DRIVE__\\__DATASET__\\__Dummy Data__\\Basic Data Types\\_Datetime.csv',
 'E:\\__ENGINEER_DRIVE__\\__DATASET__\\__Dummy Data__\\Basic Data Types\\_Datetime.json',
 'E:\\__ENGINEER_DRIVE__\\__DATASET__\\__Dummy Data__\\Basic Data Types\\_Number.csv',
 'E:\\__ENGINEER_DRIVE__\\__DATASET__\\__Dummy Data__\\Basic Data Types\\_String.csv',
 'E:\\__ENGINEER_DRIVE__\\__DATASET__\\__Dummy Data__\\Data Ingestion\\Data Engineer.ipynb',
 'E:\\__ENGINEER_DRIVE__\\__DATASET__\\__Dummy Data__\\Data Ingestion\\DummyData.json',
 'E:\\__ENGINEER_DRIVE__\\__DATASET__\\__Dummy Data__\\Data Ingestion\\sample01.csv',
 'E:\\__ENGINEER_DRIVE__\\__DATASET__\\__Dummy Data__\\Data Ingestion\\sample01.json',
 'E:\\__ENGINEER_DRIVE__\

#### [5.2.1] 멀티 파일 업로드 | 자동 경로 설정

In [28]:
# 멀티 파일 업로드 | 자동 경로 설정
import pprint 
pp = pprint.PrettyPrinter(indent=2)
api = ObjectStorage()
res_li = api.upload_files(resource_uuid=resource_uuid, files=upload_files, path=None)
pp.pprint(res_li)

Uploading: 100%|██████████| 50/50 [00:01<00:00, 25.11it/s]

[ <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>]


#### [5.2.2] 멀티 파일 업로드 | 수동 경로 설정

In [29]:
# 멀티 파일 업로드 | 수동 경로 설정
api = ObjectStorage()
res_li = api.upload_files(resource_uuid=resource_uuid, files=upload_files, path="Multi/")
pp.pprint(res_li)

Uploading: 100%|██████████| 50/50 [00:02<00:00, 23.77it/s] 

[ <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>]


### [5.3] LCA 예시

#### [5.3.1] 타겟 스토리지 찾기

In [30]:
# 스토리지 리스트 
df 

,id,resourceId,projectId,resourceUUID,workspaceId,name,creator,kind,type,created,lastModified,permissions
0,0add512f-3759-487b-aa40-9c5019874ea3,20241217_16:00_dsjeon_storage,dp-global-000000,0add512f-3759-487b-aa40-9c5019874ea3,dw-global-000000-default,20241217_16:00_dsjeon_storage,jeondongsoo,Storage,ObjectStorage,1734418711011,1734418711007,"{'read': True, 'write': True, 'execute': True}"
1,7118465e-739b-4746-8fc7-657755764ee7,240129_part_information_OU3,dp-global-000000,7118465e-739b-4746-8fc7-657755764ee7,dw-global-000000-default,240129_part_information_OU3,N/A,Storage,IndexUnit,1707553208946,1745657169549,"{'read': True, 'write': True, 'execute': True}"
2,R1132100527066_3DS_AdditiveManufacturingQualit...,3DS_AdditiveManufacturingQualityAnalyst_Data,ca221e4a-46b1-43b1-9c3e-4eea8a0aa662,R1132100527066_3DS_AdditiveManufacturingQualit...,ca221e4a-46b1-43b1-9c3e-4eea8a0aa662,3DS_AdditiveManufacturingQualityAnalyst_Data,svc_sgi_clouduser,Storage,ObjectStorage,1751697876373,1751697876334,"{'read': True, 'write': True, 'execute': True}"
3,R1132100527066_3DS_AdditiveManufacturingQualit...,3DS_AdditiveManufacturingQualityAnalyst_Image,ca221e4a-46b1-43b1-9c3e-4eea8a0aa662,R1132100527066_3DS_AdditiveManufacturingQualit...,ca221e4a-46b1-43b1-9c3e-4eea8a0aa662,3DS_AdditiveManufacturingQualityAnalyst_Image,svc_sgi_clouduser,Storage,ObjectStorage,1751697876576,1751697876537,"{'read': True, 'write': True, 'execute': True}"
4,R1132100527066_3DS_AdditiveManufacturingQualit...,3DS_AdditiveManufacturingQualityAnalyst_Result,ca221e4a-46b1-43b1-9c3e-4eea8a0aa662,R1132100527066_3DS_AdditiveManufacturingQualit...,ca221e4a-46b1-43b1-9c3e-4eea8a0aa662,3DS_AdditiveManufacturingQualityAnalyst_Result,svc_sgi_clouduser,Storage,ObjectStorage,1751697876473,1751697876435,"{'read': True, 'write': True, 'execute': True}"
...,...,...,...,...,...,...,...,...,...,...,...,...
152,5841fd29-a3a3-4780-8343-b1fe0ccaae71,r2vsurvey,dp-r2vsurvey-000000,5841fd29-a3a3-4780-8343-b1fe0ccaae71,dw-r2vsurvey-000000-default,r2vsurvey,svc_sgi_clouduser,Storage,IndexUnit,1745666684450,1751707573630,"{'read': True, 'write': False, 'execute': False}"
153,3fbd978d-c9ce-43af-9f73-ea4aed3ab4b5,sgi,dp-global-000000,3fbd978d-c9ce-43af-9f73-ea4aed3ab4b5,dw-global-000000-default,sgi,jihunyoon,Storage,IndexUnit,1734678234116,1734678234102,"{'read': True, 'write': True, 'execute': True}"
154,b35600d0-6c8d-4142-9e48-d35ab335d62b,sgi_test_syyou,dp-global-000000,b35600d0-6c8d-4142-9e48-d35ab335d62b,dw-global-000000-default,sgi_test_syyou,mobilevm,Storage,IndexUnit,1734483191336,1734483191331,"{'read': True, 'write': True, 'execute': True}"
155,595464cb-65bd-44d9-b719-c08b1a605070,syyou_storage,dp-global-000000,595464cb-65bd-44d9-b719-c08b1a605070,dw-global-000000-default,syyou_storage,mobilevm,Storage,ObjectStorage,1734677241229,1734677241224,"{'read': True, 'write': True, 'execute': True}"


In [31]:
from dserestapi import Storages, ObjectStorage
api = Storages()
res = api.search_by_name(name="SustainabilityMetricStorage", workspace_id="dp-sustainabilityexternaldata-000000")
res.json()

{'cards': [{'id': 'R1132100527066_SustainabilityMetricStorage',
   'resourceId': 'SustainabilityMetricStorage',
   'projectId': 'dp-sustainabilityexternaldata-000000',
   'resourceUUID': 'R1132100527066_SustainabilityMetricStorage',
   'workspaceId': 'dp-sustainabilityexternaldata-000000',
   'name': 'SustainabilityMetricStorage',
   'creator': 'svc_sgi_clouduser',
   'kind': 'Storage',
   'type': 'ObjectStorage',
   'created': 1751697855889,
   'lastModified': 1751697854253,
   'permissions': {'read': True, 'write': True, 'execute': True}}]}

#### [5.3.2] 특정 경로에 파일 업로드

In [32]:
api = ObjectStorage()
res = api.upload(
    resource_uuid=res.json()["cards"][0]["resourceUUID"], 
    file=r"F:\pypjts\DSE-REST-API\How to use APIs\01_Storages\Import the Storage.json", 
    path="metric/DemoObject.CreatedByRestAPI/"
)
res 

<Response [200]>

# **********************************************************************

# [6] 사용 예제 | Semantic Graph Index Unit

### [6.1] SGI Storage 신규 생성시 데이터 모델링을 동시에 수행 

In [34]:
# 데이터 모델링 
data_modeling_config = {
    "datamodel": {
        "classes": [
            {
                "name": "DummyData_01",
                "parents": [],
                "pkg": "Rawdata",
                "attributes": [
                    {
                        "name": "seq",
                        "type": {
                            "dataType": "Integer",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    },
                    {
                        "name": "title",
                        "type": {
                            "dataType": "String",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    },
                    {
                        "name": "_text",
                        "type": {
                            "dataType": "String",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    }
                ]
            },
            {
                "name": "DummyData_02",
                "parents": [],
                "pkg": "Rawdata",
                "attributes": [
                    {
                        "name": "seq",
                        "type": {
                            "dataType": "Integer",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    },
                    {
                        "name": "title",
                        "type": {
                            "dataType": "String",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    },
                    {
                        "name": "_text",
                        "type": {
                            "dataType": "String",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    }
                ]
            }
        ]
    }
}

In [35]:
from dserestapi import Storages
# 스토리지 생성 | 동시에 데이터 모델링
api = Storages()
res = api.create(
    stype="IndexUnit",
    name="DemoSGI_CreatedByRestAPI",
    description="SGI 데이터 모델링 테스트 후 삭제할 것",
    config=data_modeling_config
)
res

<Response [201]>

In [36]:
file = os.path.join(os.getcwd(), "03_SemanticGraphIndex", "Create A SGI.json")
os.makedirs(os.path.dirname(file), exist_ok=True)
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

### [6.2] 데이터 모델 스키마에 맞는 데이터 생성 후 스키마에 맞는 데이터 타입인지 정합성 체크 

In [37]:
resource_uuid = res.json()["resourceUUID"]
resource_uuid

'7d48e957-f93d-4c53-9018-e02338715286'

In [38]:
# 더미 데이터 생성 
data = [
    {
        "uri": "Rawdata.DummyData_01.0001",
        "class": "Rawdata.DummyData_01",
        "seq": 1,
        "title": "This is a title-1",
        "_text": "bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla~"
    },
    {
        "uri": "Rawdata.DummyData_01.0002",
        "class": "Rawdata.DummyData_01",
        "seq": 2,
        "title": "This is a title-2",
        "_text": "(2) bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla~"
    },
]

In [39]:
# 저장할 데이터의 정합성 체크 
from dserestapi import SemanticGraphIndex
sgi = SemanticGraphIndex()
res = sgi.validateItemsEvent(resource_uuid=resource_uuid, action="AddOrReplaceItem", data=data)
res.json()

[]

In [40]:
file = os.path.join(os.getcwd(), "03_SemanticGraphIndex", "Validation Items.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

### [6.3] 생성한 SGI의 타겟 데이터 모델에 데이터를 저장

In [42]:
from dserestapi import gen_JsonEventData 
json_event_data = gen_JsonEventData(action="AddOrReplaceItem", data=data)
pp.pprint(json_event_data)

[ { 'action': 'AddOrReplaceItem',
    'item': { '_text': 'bla bla bla bla bla bla bla bla bla bla bla bla bla '
                       'bla bla bla bla bla bla bla bla bla bla bla~',
              'class': 'Rawdata.DummyData_01',
              'seq': 1,
              'title': 'This is a title-1',
              'uri': 'Rawdata.DummyData_01.0001'}},
  { 'action': 'AddOrReplaceItem',
    'item': { '_text': '(2) bla bla bla bla bla bla bla bla bla bla bla bla '
                       'bla bla bla bla bla bla bla bla bla bla bla bla~',
              'class': 'Rawdata.DummyData_01',
              'seq': 2,
              'title': 'This is a title-2',
              'uri': 'Rawdata.DummyData_01.0002'}}]


In [43]:
res = sgi.ingest(resource_uuid=resource_uuid, data=json_event_data)
res.json()

{'notifications': []}

In [44]:
file = os.path.join(os.getcwd(), "03_SemanticGraphIndex", "Ingest Data.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

- 데이터가 저장되었는지 플랫폼에서 직접확인

### [6.4] 데이터 저장 결과 알림 받기 

In [45]:
res = sgi.notification(resource_uuid=resource_uuid)
res.json()

{'notifications': [], 'count': 0}

### [6.5] 저장한 데이터의 object 별 uri 가져오기

In [46]:
res = sgi.get_uri(resource_uuid=resource_uuid)
res 

<Response [200]>

In [49]:
file = os.path.join(os.getcwd(), "03_SemanticGraphIndex", "SGI URIs.json")
os.makedirs(os.path.dirname(file), exist_ok=True)
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

# **********************************************************************

# [7] 생각해 볼 문제 : Data Security는???
- REST API 로 Object/SGI Storage 를 생성할 경우 글로벌 워크스페이스에 노출되며 프라이빗 워크스페이스에 생성하는 방법은 현재 없다.
- 해결 방법은? 수동 + REST API를 조합해서 사용하면 된다.

1. Data Project 수동 생성 "JLE69 Knowledge Sharing"
2. Data Workspace 수동 생성 "Demospace"
3. SGI 수동 생성 "DataLake"

### [7.1] 수동으로 생성한 SGI 메타데이터 가져오기

In [50]:
from dserestapi import Storages 
storages = Storages()
res = storages.search_by_name(name="DataLake", workspace_id="dw-jle69knowledgesharing-652245-demospace")
res.json()

{'cards': [{'id': '7774d150-ce0d-47e0-b6d7-98da044b25ce',
   'resourceId': 'DataLake',
   'projectId': 'dp-jle69knowledgesharing-652245',
   'resourceUUID': '7774d150-ce0d-47e0-b6d7-98da044b25ce',
   'workspaceId': 'dw-jle69knowledgesharing-652245-demospace',
   'name': 'DataLake',
   'creator': 'jle69_gmail',
   'kind': 'Storage',
   'type': 'IndexUnit',
   'created': 1754031412415,
   'lastModified': 1754031412418,
   'permissions': {'read': True, 'write': True, 'execute': True}}]}

In [51]:
resource_uuid = res.json()["cards"][0]["resourceUUID"]

### [7.2] 더미 클래스/데이터 생성

In [53]:
import pprint 
pp = pprint.PrettyPrinter(indent=2)
from dserestapi import SGIModel
model = SGIModel(pkg_name="ModelGenerator", cls_name="TestModel")
model.gen_class()
model.gen_property(name="column_1", dtype="String")
model.gen_property(name="column_2", dtype="Integer")
model.gen_property(name="column_3", dtype="Float")
pp.pprint(model.schema)

{ 'action': 'AddOrReplaceClass',
  'classDefinition': { 'abstract': False,
                       'annotations': [],
                       'attributes': [ { 'annotations': [],
                                         'name': 'column_1',
                                         'type': 'String'},
                                       { 'annotations': [],
                                         'name': 'column_2',
                                         'type': 'Integer'},
                                       { 'annotations': [],
                                         'name': 'column_3',
                                         'type': 'Float'}],
                       'name': 'ModelGenerator.TestModel',
                       'parents': ['core.Item']}}


In [56]:
pp.pprint(model.__dict__)

{ '_cls_info': { 'action': 'AddOrReplaceClass',
                 'classDefinition': { 'abstract': False,
                                      'annotations': [],
                                      'attributes': [ { 'annotations': [],
                                                        'name': 'column_1',
                                                        'type': 'String'},
                                                      { 'annotations': [],
                                                        'name': 'column_2',
                                                        'type': 'Integer'},
                                                      { 'annotations': [],
                                                        'name': 'column_3',
                                                        'type': 'Float'}],
                                      'name': 'ModelGenerator.TestModel',
                                      'parents': ['core.Item']}},
  '_item_infos': [ 

In [55]:
model.gen_items(data=[
    {"column_1": "value_1", "column_2": 100, "column_3": 1.5},
    {"column_1": "value_2", "column_2": 200, "column_3": 2.5},
    {"column_1": "value_3", "column_2": 300, "column_3": 3.5}
])

In [57]:
conf_data = model.gen_JsonEventData()
pp.pprint(conf_data)

[ { 'action': 'AddOrReplaceClass',
    'classDefinition': { 'abstract': False,
                         'annotations': [],
                         'attributes': [ { 'annotations': [],
                                           'name': 'column_1',
                                           'type': 'String'},
                                         { 'annotations': [],
                                           'name': 'column_2',
                                           'type': 'Integer'},
                                         { 'annotations': [],
                                           'name': 'column_3',
                                           'type': 'Float'}],
                         'name': 'ModelGenerator.TestModel',
                         'parents': ['core.Item']}},
  { 'action': 'AddOrReplaceItem',
    'item': { 'class': 'ModelGenerator.TestModel',
              'column_1': 'value_1',
              'column_2': 100,
              'column_3': 1.5,
              'ur

### [7.3] 더미 클래스/데이터를 SGI에 적용 및 저장

In [58]:
from dserestapi import SemanticGraphIndex
sgi = SemanticGraphIndex()
res = sgi.ingest(resource_uuid=resource_uuid, data=conf_data)
res.json()

{'notifications': []}

# THE END.